In [4]:
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
business = []
with open("../dataset/business_all.json", "r") as file:
    for line in file:
        business.append(json.loads(line))

checkin = []
with open("../dataset/checkin_all.json", "r") as file:
    for line in file:
        checkin.append(json.loads(line))

train_review = []
with open("../dataset/review_train.json", "r") as file:
    for line in file:
        train_review.append(json.loads(line))

test_review = []
with open("../dataset/review_test.json", "r") as file:
    for line in file:
        test_review.append(json.loads(line))

user = []
with open("../dataset/user_all.json", "r") as file:
    for line in file:
        user.append(json.loads(line))
        

In [3]:
### baseline
train_average_stars = sum(d["stars"] for d in train_review) / len(train_review)
mse = sum((d["stars"] - train_average_stars) ** 2 for d in test_review) / len(test_review)
print(mse)

1.490833578439748


In [ ]:
def prepare_data(business_data, review_data, user_data):
    # Create a list of features and targets
    X = []  # Features
    y = []  # Target (stars)

    # Iterate over the business data and combine the necessary information
    for business in business_data:
        # Get relevant business data
        business_id = business['business_id']
        review_count = business['review_count']
        business_stars = business['stars']
        latitude = business['latitude']
        longitude = business['longitude']
        categories = business['categories']
        
        # Aggregate review votes (assuming each business has multiple reviews)
        total_funny = total_useful = total_cool = 0
        for review in review_data:
            if review['business_id'] == business_id:
                total_funny += review['votes']['funny']
                total_useful += review['votes']['useful']
                total_cool += review['votes']['cool']

        # Get user data (assuming each review has a user_id)
        user_avg_stars = 0
        user_review_count = 0
        for review in review_data:
            if review['business_id'] == business_id:
                user_id = review['user_id']
                for user in user_data:
                    if user['user_id'] == user_id:
                        user_avg_stars = user['average_stars']
                        user_review_count = user['review_count']
        
        # Prepare feature vector
        feature_vector = [
            review_count,
            total_funny,
            total_useful,
            total_cool,
            user_avg_stars,
            user_review_count,
            latitude,
            longitude
        ]
        X.append(feature_vector)
        y.append(business_stars)
    
    return np.array(X), np.array(y)